<center><img src="Slide1.png" width = "800"/></center>

### <b> InSAR Coherent Change Detection over Aleppo, Syria (Sep - Oct 2016)
    
<b><i> Corey Scher<sup>1</sup>
<i>Jamon Van Den Hoek<sup>2</sup>
    
<i><font size=2> 1. Department of Earth and Environmental Sciences | Graduate Center at the City University of New York
    
<i><font size=2> 2. College of Earth, Ocean, and Atmospheric Sciences | Oregon State University

### Introduction

   In this training we will walk through an example of bi-temporal coherent change detection over Aleppo, Syria to map proxies for urban damage that occurred as a result of air bombardments in 2016. We will be using the [Alaska Satellite Facility's](https://asf.alaska.edu/) [Vertex tool](https://search.asf.alaska.edu/#/) to generate two synthetic aperture radar [interferogram](https://appliedsciences.nasa.gov/sites/default/files/Session4-SAR-English_0.pdf) (InSAR) datasets to look for anomalous decreases in InSAR measurements of [coherence](https://en.wikipedia.org/wiki/Coherence_(physics) that occurred following a series of airstrike bombardments. This workflow can be completed either using the ASF [Hyp3 Python SDK](https://hyp3-docs.asf.alaska.edu/), as we will demonstrate in this notebook, or by generating InSAR coherence images manually using the ASF Vertex [user interface](https://search.asf.alaska.edu/#/) and raster band math in a geographic information system interface, such as [QGIS](https://www.qgis.org/en/site/).

In [253]:
import os
import folium
import shapely.wkt
import pandas as pd
import asf_search as asf
from hyp3_sdk import HyP3
import shapely.geometry as shp
import matplotlib.pyplot as plt

In [657]:
#Define and area of interest as WKT over Aleppo, Syria

aoi = shapely.wkt.loads("POLYGON((37.0072 36.091, \
                        37.3326 36.091,37.3326 36.3183, \
                        37.0072 36.3183,37.0072 36.091))")

# Translate to geojson for quick addition to interactrive Folium maps
geo_j = folium.GeoJson(data=gpd.GeoSeries(aoi).to_json())

In [689]:
#import credentials from .config file
credentials = pd.read_csv('/Users/coreyscher/Documents/GitHub/arset/hyp3.config', 
                          header = None, 
                          names = ['names', 'values']).set_index('names')

In [690]:
#authenticate to Hyp3 
hyp3 = HyP3(username=credentials.loc['username'][0], 
            password=credentials.loc['password'][0])

In [487]:
wkt = "POLYGON((37.0072 36.091,37.3326 36.091,37.3326 36.3183,37.0072 36.3183,37.0072 36.091))"
results = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1], 
                         intersectsWith=wkt, 
                         maxResults=100,
                        processingLevel = 'SLC',
                        end = '2016-09-05').data


In [637]:
def addFootprint(result):
    
    props = result.properties
    date = pd.to_datetime(props['startTime'])
    footprint = shp.Polygon(result.geometry['coordinates'][0])
    fid = props['fileID']
    
    return fid, footprint

In [639]:
gdf = gpd.GeoDataFrame(list(map(addFootprint, results)), 
                       columns = ['fileID',  'geometry'])


In [656]:
mapa = folium.Map([36.207522, 37.154076],
                zoom_start=7,
                  tiles='cartodbpositron')

#Add the geo_json of our are of interest to the map
geo_j.add_to(mapa)

# Plot the first five images to choose a reference scene
gdf.iloc[0:5].explore(m = mapa, 
            column = 'fileID',
           style_kwds = {'opacity': 1, 'fill': False})

In [600]:
# Let's choose the scene acquired on August 27, 2016 as a reference

reference_scene = gdf.iloc[4]

# Print out the scene ID to make sure it's the right one!
print(reference_scene)

# We can use this ID to generate an InSAR stack with the ASF tools 
insar_stack  = asf.stack_from_id(reference_scene['fileID'])

geometry    POLYGON ((35.432823 35.691978, 35.762089 37.32...
fileID      S1A_IW_SLC__1SDV_20160821T033420_20160821T0334...
Name: 2016-08-21 03:34:20, dtype: object


In [569]:
temp_baselines = []
for i in insar_stack:
    tempBaseline = i.properties['temporalBaseline']
    temp_baselines.append(tempBaseline)
#temp_baselines

In [593]:
# Let's filter the stack for dates that are of interest for our bi-temporal coherence comparison

picks = []

for i in insar_stack:
    
    # convert the start time of acquisition (string) into a pandas datetime
    start = pd.to_datetime(i.properties['startTime'])
    
    # retrieve the temporal baseline from the search result properties
    tempBaseline = i.properties['temporalBaseline']

    # We need an interferogram from representing with a similar temporal baseline to our
    # event period, which is about nine weeks from the reference image. 
    # Let's pick an secondary image with a temporal baseline ~ -64 days
    if -74 < tempBaseline < -50:
        picks.append(i)
    
    # Now we need a secondary scene representing our "event" period of Sep-Oct, 2016
    # Let's pick an secondary image that was roughly 8 week after our August, 27 reference image
    if 60 < tempBaseline < 66:
        picks.append(i)

In [601]:
# Make sure the temporal baselines are similar

for i in picks:
    start = pd.to_datetime(i.properties['startTime'])
    dif = start - pd.to_datetime(reference_scene.name)
    print(start, dif)
    
    

2016-06-10 03:34:16 -73 days +23:59:56
2016-10-26 03:33:41 65 days 23:59:21


In [661]:
# Lets plot these on a map and see how they look, making certain we have full coverage over Aleppo

stack = gpd.GeoDataFrame(list(map(addFootprint, picks)), 
                       columns = ['fileID', 'geometry'])

mapb = folium.Map([36.207522, 37.154076],
                zoom_start=7,
                  tiles='cartodbpositron')
#Add the geo_json of our are of interest to the map
geo_j.add_to(mapb)

stack.explore(m = mapb, column = 'fileID',
           style_kwds = {'opacity': 1, 'fill': False})

In [695]:
#Prepare a list of jobs using the May 2015 scene as a reference
jobs = []
for i in picks:
    
    #We have to remove the "-SLC" string from the fileID for Hyp3 to understand what granules we want!
    
    jobs.append(hyp3.prepare_insar_job(reference_scene['fileID'].replace('-SLC', ""), 
                       i.properties['fileID'].replace('-SLC', ""), 
                       name='Aleppo', 
                       include_look_vectors=False, 
                       include_los_displacement=False, 
                       include_inc_map=False, looks='10x2', 
                       include_dem=False, 
                       include_wrapped_phase=False, 
                       apply_water_mask=False, 
                       include_displacement_maps=False))

In [694]:
reference_scene['fileID'].replace('-SLC', "")

'S1A_IW_SLC__1SDV_20160821T033420_20160821T033447_012693_013F2C_3748'

In [696]:
# Take a look at the list of job dictionaries and make sure they look OK

jobs

[{'job_parameters': {'granules': ['S1A_IW_SLC__1SDV_20160821T033420_20160821T033447_012693_013F2C_3748',
    'S1A_IW_SLC__1SDV_20160610T033416_20160610T033443_011643_011D02_6360'],
   'include_look_vectors': False,
   'include_los_displacement': False,
   'include_inc_map': False,
   'looks': '10x2',
   'include_dem': False,
   'include_wrapped_phase': False,
   'apply_water_mask': False,
   'include_displacement_maps': False},
  'job_type': 'INSAR_GAMMA',
  'name': 'Aleppo'},
 {'job_parameters': {'granules': ['S1A_IW_SLC__1SDV_20160821T033420_20160821T033447_012693_013F2C_3748',
    'S1B_IW_SLC__1SDV_20161026T033341_20161026T033408_002672_00484D_A9D0'],
   'include_look_vectors': False,
   'include_los_displacement': False,
   'include_inc_map': False,
   'looks': '10x2',
   'include_dem': False,
   'include_wrapped_phase': False,
   'apply_water_mask': False,
   'include_displacement_maps': False},
  'job_type': 'INSAR_GAMMA',
  'name': 'Aleppo'}]

In [697]:
# These jobs look good so we are going to submit the list!

batch = hyp3.submit_prepared_jobs(jobs)

In [ ]:
# Now we can use the .watch() function to see the job status

hyp3.watch(batch)

  0%|                                                                      | 0/2 [timeout in 10800s]